In [1]:
import cv2
from PIL import Image, ImageOps
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

In [2]:
file=Path("./testfile.jpg")

In [ ]:
img=Image.open(file)
# 
RATIO = 1 # 原來的 ?? 倍
if RATIO < 1:
    img = img.resize((int(img.size[0]*RATIO),int(img.size[1]*RATIO)))

grey_img = ImageOps.grayscale(img)
img_nd, grey_img_nd = np.array(img), np.array(grey_img)

D:\Users\Anicca\anaconda3\envs\fileBinerlize\lib\site-packages\PIL\Image.py:3035: DecompressionBombWarning: Image size (143197800 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [ ]:
img = ImageOps.autocontrast(img)

In [ ]:
# 帶
ND_IMAGE = img_nd

# hist 預設 只統計 10 個區間(bins)
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,4))
#
axs[0].set_title("linear-y")
n, _, patches =axs[0].hist(ND_IMAGE.reshape(-1), bins=256);
# 
axs[1].set_title("log-y")
n, _, patches =axs[1].hist(ND_IMAGE.reshape(-1), bins=256);
plt.yscale('log')
#
plt.suptitle("Grey Histogram", fontsize=18)
plt.tight_layout()
plt.show()

In [ ]:
r = img.histogram()[0:0+256]
g = img.histogram()[256:256+256]
b = img.histogram()[512:512+256]

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
axs[0].plot(r, color='#FF0000')
axs[0].plot(g, color='#00FF00')
axs[0].plot(b, color='#0000FF')
#
axs[1].plot(r, color='#FF0000')
axs[1].plot(g, color='#00FF00')
axs[1].plot(b, color='#0000FF')
axs[1].axvline(60)
axs[1].axvline(210)
#axs[1].axvline(230)
plt.yscale('log')
#
plt.suptitle("RGB Histogram (plot)", fontsize=18)
#
plt.show()

In [ ]:
img_nd_copy = np.array(img_nd)
#shape_ori = img_nd_copy.shape
#img_nd_copy = img_nd_copy.reshape(-1, 3)
#
out_ = 60 
out_lt = 210

#
edit_low = img_nd_copy<out_  # 小於 下區間的
edit_height = img_nd_copy>out_lt # 大於 上區間的

edit_low = np.all(edit_low, axis=2)
edit_height = np.all(edit_height, axis=2)

img_nd_copy[edit_low] = [0,0,0]
img_nd_copy[edit_height] = [255, 255, 255]
# for idx,(r,g,b) in enumerate(img_nd_copy):
#     if r < out_ and g < out_ and b < out_:
#         img_nd_copy[idx] = [0, 0, 0]
#     if r > out_lt and g > out_lt and b > out_lt:
#         img_nd_copy[idx] = [255, 255, 255]
#
#img_nd_copy = img_nd_copy.reshape(shape_ori)

In [ ]:
plt.imshow(img_nd_copy)
plt.show()

for_save = Image.fromarray(img_nd_copy)
for_save.save("./re.png")

In [6]:
img_nd_copy.shape


NameError: name 'img_nd_copy' is not defined

In [ ]:
median_ksize = 3
for i in range(8):
    ks_ = (median_ksize,median_ksize)
    #
    after_median = ndimage.median_filter(img_nd_copy[:,:,0], size=ks_)
    after_median_1=ndimage.median_filter(img_nd_copy[:,:,1], size=ks_)
    after_median_2=ndimage.median_filter(img_nd_copy[:,:,2], size=ks_)
    #
    new_median = np.dstack((after_median,after_median_1,after_median_2))
    #
    for_save = Image.fromarray(new_median)
    for_save.save(f"./new_median_{median_ksize}.png")
    
    median_ksize += 2

In [ ]:
# make margin while
# 讓4邊緣 bang 白
w_remain = [0.0745, 0.880]
h_remain = [0.0450, 0.925]

w, h = new_median.shape[1::-1]

new_median[::, 0:int(w*w_remain[0])] = [255,255,255]  # L
new_median[::, int(w*w_remain[1]): ] = [255,255,255]  # R
new_median[0:int(h*h_remain[0]), ::] = [255,255,255]  # T
new_median[int(h*h_remain[1]): , ::] = [255,255,255]  # B

In [ ]:
for_save = Image.fromarray(new_median)
for_save.save(f"./new_median_removeMargin_{median_ksize}.png")